# Measurement of an Impulse Response with a Microphone

This notebook sketches how to measure impulse responses with a single microphone.

* Two sweeps are played, one for the loudspeaker and one for the analog feedback. 
* Two channels are recorded, the microphone and the analog feedback.

In [ ]:
# Automatically reload the acoustics_hardware package if anything is changed
%load_ext autoreload
%autoreload 2

# Load requirements
from pathlib import Path
from sys import exit
from time import sleep

import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd
from IPython.display import Audio, display, Markdown
from numpy.fft import rfft, rfftfreq
from scipy.io import savemat
from scipy.signal.waveforms import chirp

from acoustics_hardware.processors import deconvolve

## Set up audio hardware

In [ ]:
# Show list of available audio devices
print(sd.query_devices())

# Audio device configuration
device_name = 'Orion 32'
fs = 48000 # in Hz
# output channel mapping (first channel in list will be used as loopback)
device_output_ch = [1, 2] # IDs starting from 1
# input channel mapping (first channel in list will be used as loopback)
device_input_ch  = [1, 2] # IDs starting from 1

print(f'\nCheck audio device "{device_name}" ... ', end='')
try:
    sd.check_output_settings(
        device=device_name,
        channels=len(device_output_ch),
        samplerate=fs,
    )
    sd.check_input_settings(
        device=device_name,
        channels=len(device_input_ch),
        samplerate=fs,
    )
except (ValueError, sd.PortAudioError) as e:
    exit(e)
print('Done.')

# Sweep configuration
sweep_amplitude = -20         # in dB_FS
sweep_duration = 3.0          # in s
sweep_hp, sweep_lp = 50, 22e3 # in Hz

print(f'\nGenerate sweep ... ', end='')
data_out = chirp(
    t=np.arange(np.ceil(sweep_duration * fs)) / fs, # in s
    f0=sweep_hp,       # in Hz
    t1=sweep_duration, # in s
    f1=sweep_lp,       # in Hz
    method='logarithmic',
    phi=90,            # in deg
)                      # as [samples,]
data_out *= 10**(sweep_amplitude / 20) # as factor
print('Done.')

## Prepare functions for plotting

In [ ]:
def print_levels(data):
    data_rms = np.sqrt(np.mean(np.power(data, 2), axis=-1)) # linear
    data_rms = 20 * np.log10(np.abs(data_rms))              # in dB
    print(f'PEAK {20 * np.log10(np.max(np.abs(data))):+.1f} dB, '
          f'RMS {np.array2string(data_rms, precision=1, sign="+", separator=", ")} dB')

# noinspection PyShadowingNames
def plot_data(data, fs, ch_labels=None, is_stacked=False):
    data = np.atleast_2d(data)
    n_ch, n_sample = data.shape
    if ch_labels is not None and len(ch_labels) != n_ch:
        exit(f'Mismatch in number of audio channels ({n_ch}) '
             f'and number of provided labels {len(ch_labels)}.')
        
    t_data = np.linspace(0, n_sample / fs, n_sample)
    freq_data = 20*np.log10(abs(rfft(data)))
    f = rfftfreq(n_sample, 1 / fs)
    
    if is_stacked: n_ch = 1
        
    fig, axes = plt.subplots(
        nrows=n_ch,
        ncols=2 if is_stacked else 3,
        squeeze=False,
        sharex='col',
        figsize=(15, 5 if is_stacked else(3 * n_ch))
    )
    for ch in np.arange(n_ch): # individual channels
        ch_plot = (range(data.shape[0]) if is_stacked
                   else ch)
        
        # Time domain
        ax = axes[ch, 0]
        ax.plot(t_data, data[ch_plot, :].T)
        ax.set_xlim(0, n_sample / fs) # in s
        ax.grid()
        if ch_labels is not None:
            label = ([f'in_{l}' for l in ch_labels] if is_stacked 
                     else [f'in_{ch_labels[ch]}'])
            ax.legend(label, loc='upper right')
        ax.set_xlabel('Time (s)' if ch >= n_ch - 1 else '')
        ax.set_ylabel('Amplitude')

        # Frequency domain
        y_max = np.ceil(freq_data[ch_plot, :].max() / 5) * 5
        ax = axes[ch, 1]
        ax.semilogx(f, freq_data[ch_plot, :].T)
        ax.set_xlim(20, fs/2) # in Hz
        ax.set_ylim(y_max - (120 if is_stacked else 80), y_max) # in dB
        ax.grid()
        ax.set_xlabel('Frequency (Hz)' if ch >= n_ch - 1 else '')
        ax.set_ylabel('Magnitude (dB)')
        
        if not is_stacked:
            # Spectrogram
            ax = axes[ch, 2]
            ax.specgram(
                x=data[ch_plot, :].T,
                Fs=fs,        # in Hz
                NFFT=1024,    # in samples
                noverlap=512, # in samples
                mode='magnitude',
                scale='dB',
            )
            ax.set_yscale('log')
            ax.set_ylim(100, fs/2) # in Hz
            ax.set_xlabel('Time (s)' if ch >= n_ch - 1 else '')
            ax.set_ylabel('Frequency (Hz)')
    
    plt.show()

## Perform measurement

In [ ]:
# Measurement configuration
out_file_name = 'data/recorded_data.mat'
deconv_ir_hp = 20  # in Hz
deconv_ir_len = fs # in samples

# Create output data path if it does not exist
Path(out_file_name).parent.mkdir(parents=True, exist_ok=True)

# Wait before starting the measurement
sleep(1) # in s

# Start playback and recording
print(f'\nRecord sweep ... ', end='')
data_rec = sd.playrec(
    data=data_out,
    samplerate=fs,                   # in Hz
    device=device_name,
    input_mapping=device_input_ch,   # channel numbers start from 1
    output_mapping=device_output_ch, # channel numbers start from 1
    blocking=True,
).T                                  # as [channels, samples]
print('Done.')

# Print recorded peak and individual RMS levels
print_levels(data_rec)

# Show recording audio preview
for ch in range(data_rec.shape[0]):
    display(
        Markdown(f'<h3>Channel in_{device_input_ch[ch]}</h3>'),
        Audio(data_rec[ch, :], rate=fs),
    )

# Plot recorded data
plot_data(data_rec, fs, ch_labels=device_input_ch)

# Deconvolve data
data_ir = deconvolve(
    input=data_rec[0, :], # first channel as loopback
    output=data_rec[1:, :],
    fs=fs,                # in Hz
    f_low=deconv_ir_hp,   # in Hz
    f_high=sweep_lp,      # in Hz
    T=deconv_ir_len / fs, # in s
)                         # as [channels, samples]
if not data_ir.shape[0]:
    exit('No deconvolved data available (only loopback channel recorded).')
    
# Plot IR data
display(Markdown(f'<h2><center>Deconvolved Impulse Responses</h2>'))
plot_data(data_ir, fs, ch_labels=device_input_ch[1:])
plot_data(data_ir, fs, ch_labels=device_input_ch[1:], is_stacked=True)

# Store data
print(f'Save data to {out_file_name} ... ', end='')
savemat(
    out_file_name,
    {
        'signal': data_rec.T,          # as [samples, channels]
        'ir': data_ir.T,               # as [samples, channels]
        'fs': fs,                      # in Hz
        'ch_output': device_output_ch, # as list
        'ch_input': device_input_ch,   # as list
    },
)
print('Done.')